In [1]:
from sklearn.datasets import load_files       
from keras.utils import np_utils
import numpy as np
from glob import glob
from sklearn.model_selection import train_test_split


def load_dataset(path):
    data = load_files(path)
    files = np.array(data['filenames'])
    targets = np_utils.to_categorical(np.array(data['target']), 10)
    return files, targets


train_files, train_targets = load_dataset('/kaggle/input/distracteddriverdetection/data/train')

test_files,test_targets= load_dataset('/kaggle/input/distracteddriverdetection/data/test')



names = [item[17:19] for item in sorted(glob("/kaggle/input/distracteddriverdetection/data/train/*/"))]


train_files, valid_files, train_targets, valid_targets = train_test_split(train_files, train_targets, test_size=0.2, random_state=42)


print('There are %s total images.\n' % len(np.hstack([train_files, valid_files, test_files])))
print('There are %d training images.' % len(train_files))
print('There are %d total training categories.' % len(names))
print('There are %d validation images.' % len(valid_files))
print('There are %d test images.'% len(test_files))

Using TensorFlow backend.


There are 4662 total images.

There are 2484 training images.
There are 10 total training categories.
There are 622 validation images.
There are 1556 test images.


In [2]:
from keras.preprocessing import image                  
from tqdm import tqdm

def path_to_tensor(img_path):

    img = image.load_img(img_path, target_size=(224, 224))

    x = image.img_to_array(img)

    return np.expand_dims(x, axis=0)

def paths_to_tensor(img_paths):
    list_of_tensors = [path_to_tensor(img_path) for img_path in tqdm(img_paths)]
    return np.vstack(list_of_tensors)



In [3]:
from PIL import ImageFile                            
ImageFile.LOAD_TRUNCATED_IMAGES = True                 


train_tensors = paths_to_tensor(train_files).astype('float32')/255 - 0.5
valid_tensors = paths_to_tensor(valid_files).astype('float32')/255 - 0.5
test_tensors = paths_to_tensor(test_files).astype('float32')/255 - 0.5


100%|██████████| 1556/1556 [00:09<00:00, 163.60it/s]


In [4]:
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.layers import Dropout, Flatten, Dense
from keras.models import Sequential
from keras.applications.vgg16 import VGG16
model = VGG16(include_top=False)
model.summary()


58892288/58889256 [==============================] - 2s 0us/step
Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, None, None, 3)     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, None, None, 64)    1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, None, None, 64)    36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, None, None, 64)    0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, None, None, 128)   73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, None, None, 128)   147584    
______________________________________________________________

In [5]:
bottleneck_features_train_VGG16 = np.asarray([model.predict(np.expand_dims(tensor, axis=0))[0] for tensor in train_tensors],dtype=np.float32)

np.save(open('/kaggle/working/bottleneck_features_train_VGG16.npy', 'wb'),bottleneck_features_train_VGG16)


bottleneck_features_valid_VGG16 = np.asarray([model.predict(np.expand_dims(tensor, axis=0))[0] for tensor in valid_tensors],dtype=np.float32)

np.save(open('/kaggle/working/bottleneck_features_valid_VGG16.npy', 'wb'),bottleneck_features_valid_VGG16)


bottleneck_features_test_VGG16 = np.asarray([model.predict(np.expand_dims(tensor, axis=0))[0] for tensor in test_tensors],dtype=np.float32)
np.save(open('/kaggle/working/bottleneck_features_test_VGG16.npy', 'wb'),bottleneck_features_test_VGG16)


In [6]:
print(bottleneck_features_train_VGG16.shape)
print(bottleneck_features_valid_VGG16.shape)
print(bottleneck_features_test_VGG16.shape)


(2484, 7, 7, 512)
(622, 7, 7, 512)
(1556, 7, 7, 512)


In [7]:
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.layers import Dropout, Flatten, Dense
from keras.models import Sequential
VGG16_model = Sequential()
VGG16_model.add(GlobalAveragePooling2D(input_shape=bottleneck_features_train_VGG16.shape[1:]))
VGG16_model.add(Dense(10, activation='softmax', kernel_initializer='glorot_normal'))

VGG16_model.summary()


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
global_average_pooling2d_1 ( (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                5130      
Total params: 5,130
Trainable params: 5,130
Non-trainable params: 0
_________________________________________________________________


In [8]:
VGG16_model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

In [9]:
from keras.callbacks import ModelCheckpoint
checkpointer = ModelCheckpoint(filepath='/kaggle/working/weights.best.VGG16.hdf5_transfer_learning', 
                               verbose=1, save_best_only=True)

VGG16_model.fit(bottleneck_features_train_VGG16, train_targets, 
          validation_data=(bottleneck_features_valid_VGG16, valid_targets),
          epochs=400, batch_size=16, callbacks=[checkpointer], verbose=1)


Train on 2484 samples, validate on 622 samples
Epoch 1/400
2484/2484 [==============================] - 1s 222us/step - loss: 2.2828 - accuracy: 0.1473 - val_loss: 2.2389 - val_accuracy: 0.1222

Epoch 00001: val_loss improved from inf to 2.23888, saving model to /kaggle/working/weights.best.VGG16.hdf5_transfer_learning
Epoch 2/400
2484/2484 [==============================] - 0s 143us/step - loss: 2.1963 - accuracy: 0.2395 - val_loss: 2.1527 - val_accuracy: 0.2733

Epoch 00002: val_loss improved from 2.23888 to 2.15267, saving model to /kaggle/working/weights.best.VGG16.hdf5_transfer_learning
Epoch 3/400
2484/2484 [==============================] - 0s 156us/step - loss: 2.1125 - accuracy: 0.3257 - val_loss: 2.0942 - val_accuracy: 0.3376

Epoch 00003: val_loss improved from 2.15267 to 2.09422, saving model to /kaggle/working/weights.best.VGG16.hdf5_transfer_learning
Epoch 4/400
2484/2484 [==============================] - 0s 152us/step - loss: 2.0390 - accuracy: 0.3998 - val_loss: 2.0290

2484/2484 [==============================] - 0s 157us/step - loss: 1.0578 - accuracy: 0.8015 - val_loss: 1.1586 - val_accuracy: 0.7235

Epoch 00030: val_loss improved from 1.16984 to 1.15861, saving model to /kaggle/working/weights.best.VGG16.hdf5_transfer_learning
Epoch 31/400
2484/2484 [==============================] - 0s 166us/step - loss: 1.0374 - accuracy: 0.8039 - val_loss: 1.1398 - val_accuracy: 0.7154

Epoch 00031: val_loss improved from 1.15861 to 1.13975, saving model to /kaggle/working/weights.best.VGG16.hdf5_transfer_learning
Epoch 32/400
2484/2484 [==============================] - 0s 156us/step - loss: 1.0161 - accuracy: 0.8060 - val_loss: 1.1214 - val_accuracy: 0.7331

Epoch 00032: val_loss improved from 1.13975 to 1.12137, saving model to /kaggle/working/weights.best.VGG16.hdf5_transfer_learning
Epoch 33/400
2484/2484 [==============================] - 0s 148us/step - loss: 0.9984 - accuracy: 0.8080 - val_loss: 1.1192 - val_accuracy: 0.7186

Epoch 00033: val_loss impro

2484/2484 [==============================] - 0s 149us/step - loss: 0.6414 - accuracy: 0.8909 - val_loss: 0.7680 - val_accuracy: 0.8215

Epoch 00062: val_loss improved from 0.79120 to 0.76798, saving model to /kaggle/working/weights.best.VGG16.hdf5_transfer_learning
Epoch 63/400
2484/2484 [==============================] - 0s 146us/step - loss: 0.6327 - accuracy: 0.8937 - val_loss: 0.7640 - val_accuracy: 0.8296

Epoch 00063: val_loss improved from 0.76798 to 0.76398, saving model to /kaggle/working/weights.best.VGG16.hdf5_transfer_learning
Epoch 64/400
2484/2484 [==============================] - 0s 156us/step - loss: 0.6252 - accuracy: 0.8909 - val_loss: 0.7533 - val_accuracy: 0.8151

Epoch 00064: val_loss improved from 0.76398 to 0.75325, saving model to /kaggle/working/weights.best.VGG16.hdf5_transfer_learning
Epoch 65/400
2484/2484 [==============================] - 0s 154us/step - loss: 0.6174 - accuracy: 0.8925 - val_loss: 0.7413 - val_accuracy: 0.8280

Epoch 00065: val_loss impro

2484/2484 [==============================] - 0s 143us/step - loss: 0.4397 - accuracy: 0.9308 - val_loss: 0.5739 - val_accuracy: 0.8633

Epoch 00095: val_loss improved from 0.57857 to 0.57394, saving model to /kaggle/working/weights.best.VGG16.hdf5_transfer_learning
Epoch 96/400
2484/2484 [==============================] - 0s 167us/step - loss: 0.4350 - accuracy: 0.9328 - val_loss: 0.5679 - val_accuracy: 0.8746

Epoch 00096: val_loss improved from 0.57394 to 0.56785, saving model to /kaggle/working/weights.best.VGG16.hdf5_transfer_learning
Epoch 97/400
2484/2484 [==============================] - 0s 164us/step - loss: 0.4306 - accuracy: 0.9336 - val_loss: 0.5702 - val_accuracy: 0.8601

Epoch 00097: val_loss did not improve from 0.56785
Epoch 98/400
2484/2484 [==============================] - 0s 158us/step - loss: 0.4280 - accuracy: 0.9320 - val_loss: 0.5573 - val_accuracy: 0.8698

Epoch 00098: val_loss improved from 0.56785 to 0.55734, saving model to /kaggle/working/weights.best.VGG16


Epoch 00129: val_loss did not improve from 0.45740
Epoch 130/400
2484/2484 [==============================] - 0s 141us/step - loss: 0.3196 - accuracy: 0.9513 - val_loss: 0.4569 - val_accuracy: 0.8859

Epoch 00130: val_loss improved from 0.45740 to 0.45688, saving model to /kaggle/working/weights.best.VGG16.hdf5_transfer_learning
Epoch 131/400
2484/2484 [==============================] - 0s 140us/step - loss: 0.3172 - accuracy: 0.9501 - val_loss: 0.4529 - val_accuracy: 0.8859

Epoch 00131: val_loss improved from 0.45688 to 0.45292, saving model to /kaggle/working/weights.best.VGG16.hdf5_transfer_learning
Epoch 132/400
2484/2484 [==============================] - 0s 152us/step - loss: 0.3154 - accuracy: 0.9513 - val_loss: 0.4526 - val_accuracy: 0.8971

Epoch 00132: val_loss improved from 0.45292 to 0.45255, saving model to /kaggle/working/weights.best.VGG16.hdf5_transfer_learning
Epoch 133/400
2484/2484 [==============================] - 0s 163us/step - loss: 0.3132 - accuracy: 0.9513 -

...

2484/2484 [==============================] - 0s 147us/step - loss: 0.1571 - accuracy: 0.9823 - val_loss: 0.2936 - val_accuracy: 0.9196

Epoch 00236: val_loss did not improve from 0.28923
Epoch 237/400
2484/2484 [==============================] - 0s 148us/step - loss: 0.1563 - accuracy: 0.9819 - val_loss: 0.2910 - val_accuracy: 0.9260

Epoch 00237: val_loss did not improve from 0.28923
Epoch 238/400
2484/2484 [==============================] - 0s 152us/step - loss: 0.1551 - accuracy: 0.9815 - val_loss: 0.2900 - val_accuracy: 0.9277

Epoch 00238: val_loss did not improve from 0.28923
Epoch 239/400
2484/2484 [==============================] - 0s 154us/step - loss: 0.1533 - accuracy: 0.9823 - val_loss: 0.2862 - val_accuracy: 0.9212

Epoch 00239: val_loss improved from 0.28923 to 0.28621, saving model to /kaggle/working/weights.best.VGG16.hdf5_transfer_learning
Epoch 240/400
2484/2484 [==============================] - 0s 142us/step - loss: 0.1533 - accuracy: 0.9835 - val_loss: 0.2880 - val


Epoch 00273: val_loss improved from 0.26029 to 0.25884, saving model to /kaggle/working/weights.best.VGG16.hdf5_transfer_learning
Epoch 274/400
2484/2484 [==============================] - 0s 155us/step - loss: 0.1273 - accuracy: 0.9867 - val_loss: 0.2754 - val_accuracy: 0.9244

Epoch 00274: val_loss did not improve from 0.25884
Epoch 275/400
2484/2484 [==============================] - 0s 151us/step - loss: 0.1278 - accuracy: 0.9871 - val_loss: 0.2665 - val_accuracy: 0.9244

Epoch 00275: val_loss did not improve from 0.25884
Epoch 276/400
2484/2484 [==============================] - 0s 147us/step - loss: 0.1266 - accuracy: 0.9879 - val_loss: 0.2603 - val_accuracy: 0.9293

Epoch 00276: val_loss did not improve from 0.25884
Epoch 277/400
2484/2484 [==============================] - 0s 144us/step - loss: 0.1251 - accuracy: 0.9879 - val_loss: 0.2664 - val_accuracy: 0.9325

Epoch 00277: val_loss did not improve from 0.25884
Epoch 278/400
2484/2484 [==============================] - 0s 142


Epoch 00310: val_loss improved from 0.24018 to 0.23965, saving model to /kaggle/working/weights.best.VGG16.hdf5_transfer_learning
Epoch 311/400
2484/2484 [==============================] - 0s 149us/step - loss: 0.1068 - accuracy: 0.9907 - val_loss: 0.2478 - val_accuracy: 0.9196

Epoch 00311: val_loss did not improve from 0.23965
Epoch 312/400
2484/2484 [==============================] - 0s 139us/step - loss: 0.1061 - accuracy: 0.9911 - val_loss: 0.2369 - val_accuracy: 0.9325

Epoch 00312: val_loss improved from 0.23965 to 0.23694, saving model to /kaggle/working/weights.best.VGG16.hdf5_transfer_learning
Epoch 313/400
2484/2484 [==============================] - 0s 140us/step - loss: 0.1048 - accuracy: 0.9907 - val_loss: 0.2455 - val_accuracy: 0.9373

Epoch 00313: val_loss did not improve from 0.23694
Epoch 314/400
2484/2484 [==============================] - 0s 144us/step - loss: 0.1053 - accuracy: 0.9924 - val_loss: 0.2358 - val_accuracy: 0.9341

Epoch 00314: val_loss improved from 0

2484/2484 [==============================] - 0s 150us/step - loss: 0.0894 - accuracy: 0.9932 - val_loss: 0.2219 - val_accuracy: 0.9373

Epoch 00349: val_loss improved from 0.22491 to 0.22191, saving model to /kaggle/working/weights.best.VGG16.hdf5_transfer_learning
Epoch 350/400
2484/2484 [==============================] - 0s 151us/step - loss: 0.0886 - accuracy: 0.9919 - val_loss: 0.2326 - val_accuracy: 0.9309

Epoch 00350: val_loss did not improve from 0.22191
Epoch 351/400
2484/2484 [==============================] - 0s 141us/step - loss: 0.0889 - accuracy: 0.9936 - val_loss: 0.2223 - val_accuracy: 0.9325

Epoch 00351: val_loss did not improve from 0.22191
Epoch 352/400
2484/2484 [==============================] - 0s 145us/step - loss: 0.0885 - accuracy: 0.9936 - val_loss: 0.2174 - val_accuracy: 0.9357

Epoch 00352: val_loss improved from 0.22191 to 0.21742, saving model to /kaggle/working/weights.best.VGG16.hdf5_transfer_learning
Epoch 353/400
2484/2484 [===========================

2484/2484 [==============================] - 0s 141us/step - loss: 0.0754 - accuracy: 0.9952 - val_loss: 0.2108 - val_accuracy: 0.9421

Epoch 00387: val_loss did not improve from 0.20730
Epoch 388/400
2484/2484 [==============================] - 0s 144us/step - loss: 0.0759 - accuracy: 0.9952 - val_loss: 0.2068 - val_accuracy: 0.9389

Epoch 00388: val_loss improved from 0.20730 to 0.20676, saving model to /kaggle/working/weights.best.VGG16.hdf5_transfer_learning
Epoch 389/400
2484/2484 [==============================] - 0s 140us/step - loss: 0.0752 - accuracy: 0.9948 - val_loss: 0.2116 - val_accuracy: 0.9325

Epoch 00389: val_loss did not improve from 0.20676
Epoch 390/400
2484/2484 [==============================] - 0s 146us/step - loss: 0.0739 - accuracy: 0.9956 - val_loss: 0.2188 - val_accuracy: 0.9277

Epoch 00390: val_loss did not improve from 0.20676
Epoch 391/400
2484/2484 [==============================] - 0s 146us/step - loss: 0.0745 - accuracy: 0.9944 - val_loss: 0.2156 - val

In [10]:
VGG16_model.load_weights('/kaggle/working/weights.best.VGG16.hdf5_transfer_learning')

In [11]:
VGG16_predictions = [VGG16_model.predict(np.expand_dims(tensor, axis=0))[0] for tensor in bottleneck_features_test_VGG16]

In [12]:
test_files_final = [item_test[15:] for item_test in test_files]
VGG16_subm = np.column_stack((np.asarray(test_files_final), np.asarray(VGG16_predictions,dtype=np.float32)))

In [13]:
print(VGG16_subm[1:10])

[['istracteddriverdetection/data/test/test/img_385.jpg' '0.058994595'
  '0.08912532' '0.34637123' '0.058494426' '0.00052492844' '0.01131132'
  '0.33978584' '0.00016164302' '0.0020132393' '0.09321744']
 ['istracteddriverdetection/data/test/test/img_1741.jpg' '0.0019679223'
  '4.73201e-05' '0.01752385' '0.18795694' '0.005123911' '0.02184142'
  '0.45006043' '0.19345342' '0.120064825' '0.001959921']
 ['istracteddriverdetection/data/test/test/img_167.jpg' '0.0005676121'
  '0.7745486' '0.0019875194' '0.14757727' '0.00276104' '2.8612158e-06'
  '0.020089475' '0.024120754' '0.020385921' '0.007959037']
 ['istracteddriverdetection/data/test/test/img_1479.jpg' '0.047218584'
  '0.0132582905' '0.00028449806' '0.010444333' '0.018999305' '0.02612158'
  '0.0018582201' '0.008073772' '0.0129927285' '0.86074877']
 ['istracteddriverdetection/data/test/test/img_1574.jpg' '0.37549126'
  '0.000106252344' '0.001356539' '0.00757148' '0.48021302' '0.06328166'
  '0.050075576' '2.8565159e-05' '0.00045775878' '0.02

In [14]:
from keras.applications.vgg16 import VGG16
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential, Model
from keras.layers import Dropout, Flatten, Dense

from keras.applications.vgg16 import VGG16
base_model = VGG16(include_top=False)
base_model.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, None, None, 3)     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, None, None, 64)    1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, None, None, 64)    36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, None, None, 64)    0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, None, None, 128)   73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, None, None, 128)   147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, None, None, 128)   0     

In [15]:
VGG16_top_model = Sequential()
VGG16_top_model.add(GlobalAveragePooling2D(input_shape=base_model.output_shape[1:]))
VGG16_top_model.add(Dense(10, activation='softmax', kernel_initializer='glorot_normal'))


VGG16_top_model.load_weights('/kaggle/working/weights.best.VGG16.hdf5_transfer_learning')

model = Model(input= base_model.input, output= VGG16_top_model(base_model.output))

VGG16_top_model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
global_average_pooling2d_2 ( (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 10)                5130      
Total params: 5,130
Trainable params: 5,130
Non-trainable params: 0
_________________________________________________________________


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:8: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("se...)`
  


In [16]:
for layer in model.layers[:15]:
    layer.trainable = False

# compile the model with a SGD/momentum optimizer
# and a very slow learning rate.
model.compile(loss='categorical_crossentropy',optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),metrics=['accuracy'])

In [17]:
from keras.callbacks import ModelCheckpoint

checkpointer = ModelCheckpoint(filepath='/kaggle/working/weights.best.VGG16.hdf5_fine_tuning', 
                               verbose=1, save_best_only=True)

model.fit(train_tensors, train_targets, 
          validation_data=(valid_tensors, valid_targets),
          epochs=10, batch_size=16, callbacks=[checkpointer], verbose=1)

Train on 2484 samples, validate on 622 samples
Epoch 1/10
2484/2484 [==============================] - 974s 392ms/step - loss: 0.2399 - accuracy: 0.9211 - val_loss: 0.1437 - val_accuracy: 0.9630

Epoch 00001: val_loss improved from inf to 0.14372, saving model to /kaggle/working/weights.best.VGG16.hdf5_fine_tuning
Epoch 2/10
2484/2484 [==============================] - 985s 396ms/step - loss: 0.0351 - accuracy: 0.9936 - val_loss: 0.1211 - val_accuracy: 0.9614

Epoch 00002: val_loss improved from 0.14372 to 0.12112, saving model to /kaggle/working/weights.best.VGG16.hdf5_fine_tuning
Epoch 3/10
2484/2484 [==============================] - 990s 398ms/step - loss: 0.0074 - accuracy: 1.0000 - val_loss: 0.0892 - val_accuracy: 0.9695

Epoch 00003: val_loss improved from 0.12112 to 0.08920, saving model to /kaggle/working/weights.best.VGG16.hdf5_fine_tuning
Epoch 4/10
2484/2484 [==============================] - 984s 396ms/step - loss: 0.0035 - accuracy: 1.0000 - val_loss: 0.0882 - val_accurac

In [18]:
model.load_weights('/kaggle/working/weights.best.VGG16.hdf5_fine_tuning')

In [19]:
VGG16_predictions_fine_tuned = [model.predict(np.expand_dims(tensor, axis=0))[0] for tensor in test_tensors]

In [20]:
VGG16_subm_fine_tuned = np.column_stack((np.asarray(test_files_final), np.asarray(VGG16_predictions_fine_tuned,dtype=np.float32)))

In [21]:
print(VGG16_subm_fine_tuned[1:10])

[['istracteddriverdetection/data/test/test/img_385.jpg' '0.08813192'
  '0.39732525' '0.061706513' '0.0062473165' '2.4957817e-05'
  '0.0022696475' '0.42528674' '4.4610547e-06' '4.105503e-05'
  '0.018962108']
 ['istracteddriverdetection/data/test/test/img_1741.jpg' '0.0070710545'
  '1.6618675e-05' '0.0035119823' '0.8582791' '0.002671449' '0.0070028724'
  '0.087300934' '0.013848256' '0.020031072' '0.00026669368']
 ['istracteddriverdetection/data/test/test/img_167.jpg' '0.0002879953'
  '0.9585907' '0.00020574874' '0.025199082' '0.00019821415'
  '2.653042e-07' '0.003826441' '0.00085477205' '0.0092280265'
  '0.00160886']
 ['istracteddriverdetection/data/test/test/img_1479.jpg' '0.019584458'
  '0.0003838691' '6.132026e-07' '0.00011226081' '0.00062465656'
  '0.1185391' '1.2245856e-05' '0.0022442462' '0.0012107447' '0.8572879']
 ['istracteddriverdetection/data/test/test/img_1574.jpg' '0.64535755'
  '7.0214046e-06' '4.172873e-05' '0.0071861963' '0.27488' '0.024316154'
  '0.0069282134' '1.2500784